In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pd.options.display.float_format = '{:.0f}'.format

<h2>Only for google colab:</h2>

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

# !ls '/content/gdrive/My Drive/Data challenge/code'

# # import single json file as dataframe
# df = pd.read_json('/content/gdrive/My Drive/Data challenge/code/airlines-1558527599826.json', orient = 'records', lines = True)

In [4]:
# df = pd.read_json('airlines-1558527599826.json', orient = 'records', lines = True)
df = pd.read_csv('Full_basic.csv', usecols=['id_str','created_at'])

C:\Users\philipp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
not_number = df[df['id_str'].isna()]
lst = not_number.index
df.drop(lst, inplace = True)
df['id_str']=df['id_str'].astype('int64')
# df['id']=df['id'].astype('int64')
df = df.rename({'id_str': 'id'}, axis=1)

In [6]:
df_conv = pd.read_csv('conversation_full.csv')

In [7]:
df['in_reply_to_status_id']=df_conv['in_reply_to_status_id']

In [8]:
df=df.sort_values('created_at')

In [9]:
# def myindex(lst, target):
#     for index, item in enumerate(lst):
#         if item == target:
#             return [index]
#         if isinstance(item, str): # or 'basestring' in 2.x
#             return []
#         try:
#             path = myindex(item, target)
#         except TypeError:
#             pass
#         else:
#             if path:
#                 return [index] + path
#     return []

In [10]:
# contain=[]
# index_length=0
# for row in df.itertuples():
    
#     if math.isnan(getattr(row,'id')):
#         print('id is nan')
#         continue
# #     print(int(getattr(row,'id')))
# #     print(type(getattr(row,'in_reply_to_status_id')))
#     if math.isnan(getattr(row,'in_reply_to_status_id')):
#         conv_list.append([int(getattr(row,'id'))])
#     else:
#         index=myindex(conv_list,int(getattr(row,'in_reply_to_status_id')))
#         if len(index)>index_length:
#             index_length=len(index)
#         if len(index)==0:
# #             print('could not match in_reply_to_status_id',int(getattr(row,'in_reply_to_status_id')))
#             continue
#         if len(index)==1:
#             if type(conv_list[index[0]]) is int:
#                 conv_list[index[0]]=[conv_list[index[0]]]
#             conv_list[index[0]].append(int(getattr(row,'id')))
#             continue
#         if len(index)==2:
#             if type(conv_list[index[0]][index[1]]) is int:
#                 conv_list[index[0]][index[1]]=[conv_list[index[0]][index[1]]]
#             conv_list[index[0]][index[1]].append(int(getattr(row,'id')))
#             continue
#         if len(index)==3:
#             if type(conv_list[index[0]][index[1]][index[2]]) is int:
#                 conv_list[index[0]][index[1]][index[2]]=[conv_list[index[0]][index[1]][index[2]]]
#             conv_list[index[0]][index[1]][index[2]].append(int(getattr(row,'id')))
#             continue
#         if len(index)==4:
#             if type(conv_list[index[0]][index[1]][index[2]][index[3]]) is int:
#                 conv_list[index[0]][index[1]][index[2]][index[3]]=[conv_list[index[0]][index[1]][index[2]][index[3]]]
#             conv_list[index[0]][index[1]][index[2]][index[3]].append(int(getattr(row,'id')))
#             continue
#         if len(index)==5:
#             if type(conv_list[index[0]][index[1]][index[2]][index[3]][index[4]]) is int:
#                 conv_list[index[0]][index[1]][index[2]][index[3]][index[4]]=[conv_list[index[0]][index[1]][index[2]][index[3]][index[4]]]
#             conv_list[index[0]][index[1]][index[2]][index[3]][index[4]].append(int(getattr(row,'id')))
#             continue
#         if len(index)==6:
#             if type(conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]]) is int:
#                 conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]]=[conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]]]
#             conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]].append(int(getattr(row,'id')))
#         if len(index)==7:
#             if type(conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]][index[6]]) is int:
#                 conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]][index[6]]=[conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]][index[6]]]
#             conv_list[index[0]][index[1]][index[2]][index[3]][index[4]][index[5]][index[6]].append(int(getattr(row,'id')))

In [11]:
df.drop_duplicates(subset ="id", 
                     keep = 'first', inplace = True)

In [12]:
df=df.drop(df[df['id']==df['in_reply_to_status_id']].index)

In [13]:
df_reply=df[~df['in_reply_to_status_id'].isnull()]
index=0
def met2():
    def lookup(ids):
        ret_list=[]
        df_np=df_reply.loc[df_reply['in_reply_to_status_id']==ids]['id'].values
        for row in range(0,len(df_np)):
            global index
            used[index]=df_np[row]
            index+=1
            buffer=df[df['id']==df_np[row]]
            if len(buffer)==0:
                ret_list.append(df_np[row])
            else:
                lookup_ret=lookup(buffer['id'].item())
                if lookup_ret:
                    ret_list.extend([df_np[row],lookup_ret])
                else:
                    ret_list.append(df_np[row])
        if len(ret_list)==0:
            return
        return ret_list
    used=np.zeros(len(df), dtype='int64')
    with open('conversations_list.txt', 'w') as f:
        id_np=np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64'))
        for row in range(0,len(id_np)):
            if math.isnan(id_np[row]):continue
            this_id=id_np[row]
            if this_id in used: continue
            bla=[this_id]
            try:
                lookup_ret=lookup(this_id)
            except:
                continue
            if lookup_ret:
                if len(lookup_ret)==1:
                    bla.append(lookup_ret[0])
                else:
                    bla.extend(lookup_ret)
                f.write("%s\n" % bla)

In [18]:
%%time
met2()

Wall time: 5h 4min 20s


In [24]:
np.concatenate([np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64')),df[~df['in_reply_to_status_id'].isnull()]['id']])

array([1131173010508062720, 1131173313982734336, 1131173345918148608, ...,
       1177008913675247616, 1177008933992402944, 1177009156261392384],
      dtype=int64)

In [25]:
pd.DataFrame(np.concatenate([np.intersect1d(df[df['in_reply_to_status_id'].isnull()]['id'], df[~df['in_reply_to_status_id'].isnull()]['in_reply_to_status_id'].astype('int64')),df[~df['in_reply_to_status_id'].isnull()]['id']]))

,0
0,1131173010508062720
1,1131173313982734336
2,1131173345918148608
3,1131174768114323456
4,1131174885865205760
...,...
2113085,1177008900912160768
2113086,1177008905513295872
2113087,1177008913675247616
2113088,1177008933992402944
